# Create a parsed morpheme list to modify (uncomment to save it)

In [ ]:
import pandas as pd

# Load the input CSV
df = pd.read_csv("morpheme_list.csv")

# List to hold each individual morpheme entry
morpheme_entries = []

# Loop through each row
for morpheme_str in df["morphemes"].dropna():
    morphemes = morpheme_str.split(";")
    for item in morphemes:
        parts = item.strip().split("/", 1)
        if len(parts) == 2:
            nahuatl, gloss = parts
            morpheme_entries.append((nahuatl.strip(), gloss.strip()))

# Convert to DataFrame and sort
morpheme_df = pd.DataFrame(morpheme_entries, columns=["Nahuatl", "English"])
morpheme_df = morpheme_df.sort_values(by="Nahuatl").reset_index(drop=True)

# Save to CSV in current working directory
#output_filename = "parsed_morphemes.csv"
#morpheme_df.to_csv(output_filename, index=False)

# Show first few rows
morpheme_df.head()


# Create labels from the modified parsed morpheme list

In [8]:
import pandas as pd

# Load both CSVs
parsed = pd.read_csv("parsed_morphemes.csv")
morpheme_list = pd.read_csv("morpheme_list.csv")

# Keep only rows in parsed with non-empty Label
parsed = parsed[parsed["Label"].notna() & (parsed["Label"].str.strip() != "")]

# Remove duplicates (some morphemes may appear multiple times with same Label)
parsed = parsed.drop_duplicates(subset=["Nahuatl", "Label"])

# Prepare a list to store matched rows
matched_rows = []

# For each unique morpheme-label pair
for _, row in parsed.iterrows():
    nahuatl = row["Nahuatl"]
    label = row["Label"]
    
    # Find rows in morpheme_list where the morphemes column contains the Nahuatl string
    mask = morpheme_list["morphemes"].fillna("").str.split(";").apply(
        lambda lst: any(nahuatl.strip() == m.strip().split("/")[0] for m in lst)
    )
    
    matches = morpheme_list[mask].copy()
    matches["Label"] = label
    
    matched_rows.append(matches)

# Concatenate all matched DataFrames
final_df = pd.concat(matched_rows, ignore_index=True)

# Optional: drop duplicates just in case (based on all columns)
final_df = final_df.drop_duplicates()

# Save to CSV
output_filename = "labeled_morpheme_list.csv"
final_df.to_csv(output_filename, index=False)

# Preview
final_df.head()


ID      official_name   type text_subchapters illustrations  \
0  ID001  Aca-capac-quilitl  plant              09e       p079_01   
1  ID002              Acatl  plant              NaN       p079_04   
2  ID005           Acxoyatl  plant              10b       p095_02   
3  ID091       Chicom-acatl  plant              09f           NaN   
4  ID302         acatzanatl   bird              09b           NaN   

                           synonyms  \
0  acacapac-quilitl; acapac-quilitl   
1                               NaN   
2                               NaN   
3                       chicom-catl   
4                               NaN   

                                           morphemes       Label  
0  Quilitl/edible green; Acatl/reed; Capa/strong ...  Acatl/reed  
1                                         Acatl/reed  Acatl/reed  
2                     Acatl/reed; Xoyatl/branch/twig  Acatl/reed  
3                          Chicome/seven; Acatl/reed  Acatl/reed  
4                           Acatl/reed; zanatl/water  Acatl/reed

In [9]:
final_df["Label"].value_counts()

Label
Xihuitl/herbs, green leaves              37
Xochitl/flower                           34
Quahuitl/tree or wood                    23
Patli/medicine                           18
Xocotl/sour fruit                        14
Atl/water                                13
Te/stone                                 10
Quilitl/edible green                      9
Iztac/white                               8
Quetzal/precious, feathered, or green     8
Tlal/earth or ground                      7
Tlaco/half or middle                      7
Teo/divine                                7
Huitz/needle or thorn                     7
Ocotl/pine                                6
Acatl/reed                                6
Zacatl/grass                              5
Tototl/bird                               5
Nochtli/prickly pear                      5
Tetl/stone                                5
Amatl/paper                               5
Copalli/incense                           4
Tzon/hair                 

# Add morpheme labels to md files to use

In [12]:
import os
import shutil
import pandas as pd
from pathlib import Path

# --- Configuration ---
input_csv = "labeled_morpheme_list.csv"
input_folders = [
    "6_chapters", "7_plants_with_illlustrations", "8_plants",
    "9_stones", "10_animals", "11_birds", "12_other"
]
output_folder = "14_all_md_files"
os.makedirs(output_folder, exist_ok=True)

# --- Load morpheme data ---
df = pd.read_csv(input_csv)
df = df[df["Label"].notna() & df["Label"].str.strip().ne("")]
df["official_name"] = df["official_name"].astype(str)
grouped_labels = df.groupby("official_name")["Label"].apply(list).to_dict()

# --- File processing ---
for folder in input_folders:
    input_path = Path(folder)
    output_path = Path(output_folder)

    for md_file in input_path.glob("*.md"):
        official_name = md_file.stem
        out_file = output_path / md_file.name

        if folder == "6_chapters":
            shutil.copy(md_file, out_file)
            continue

        with open(md_file, "r", encoding="utf-8") as f:
            lines = f.readlines()

        morphemes = grouped_labels.get(official_name)

        if morphemes:
            morpheme_block = ["\n", "**Morphemes:**\n", "\n"] + [f"- {m}\n" for m in sorted(set(morphemes))] + ["\n"]

            new_lines = []
            i = 0
            inserted = False

            while i < len(lines):
                line = lines[i]
                new_lines.append(line)

                if "**Variants:**" in line and not inserted:
                    i += 1
                    new_lines.append(lines[i])
                    # Collect bullet points after variants
                    while i + 1 < len(lines) and lines[i + 1].lstrip().startswith("-"):
                        i += 1
                        new_lines.append(lines[i])
                    new_lines.append("\n")  # Blank line after last bullet
                    new_lines.extend(morpheme_block)
                    inserted = True
                elif not inserted and line.startswith("## Subchapter"):
                    # No variants found; insert before subchapter header
                    new_lines = new_lines[:-1] + morpheme_block + [line]
                    inserted = True

                i += 1

            if not inserted:
                new_lines = morpheme_block + lines

            with open(out_file, "w", encoding="utf-8") as f:
                f.writelines(new_lines)
        else:
            shutil.copy(md_file, out_file)
